In [1]:
# Dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder


In [2]:
file_path ='resources/fifa_world_players_alt.csv'
all_fifa_data_df = pd.read_csv(file_path)
all_fifa_data_df['rating'] = (all_fifa_data_df['Potential'].astype(int) + all_fifa_data_df['Overall'].astype(int))/2
all_fifa_data_df['rating'] = all_fifa_data_df['rating'].apply(np.int64)
#Categorical Columns
all_fifa_data_df['Club'].fillna('No Club', inplace = True)
all_fifa_data_df['Position'].fillna('unknown',inplace=True)
all_fifa_data_df


,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,rating
0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,...,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M,94
1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,...,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M,94
2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,...,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M,92
3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,...,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M,92
4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,...,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,238813,J. Lundstram,19,https://cdn.sofifa.org/players/4/19/238813.png,England,https://cdn.sofifa.org/flags/14.png,47,65,Crewe Alexandra,https://cdn.sofifa.org/teams/2/light/121.png,...,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,€143K,56
18203,243165,N. Christoffersson,19,https://cdn.sofifa.org/players/4/19/243165.png,Sweden,https://cdn.sofifa.org/flags/46.png,47,63,Trelleborgs FF,https://cdn.sofifa.org/teams/2/light/703.png,...,22.0,15.0,19.0,10.0,9.0,9.0,5.0,12.0,€113K,55
18204,241638,B. Worman,16,https://cdn.sofifa.org/players/4/19/241638.png,England,https://cdn.sofifa.org/flags/14.png,47,67,Cambridge United,https://cdn.sofifa.org/teams/2/light/1944.png,...,32.0,13.0,11.0,6.0,5.0,10.0,6.0,13.0,€165K,57
18205,246268,D. Walker-Rice,17,https://cdn.sofifa.org/players/4/19/246268.png,England,https://cdn.sofifa.org/flags/14.png,47,66,Tranmere Rovers,https://cdn.sofifa.org/teams/2/light/15048.png,...,20.0,25.0,27.0,14.0,6.0,14.0,8.0,9.0,€143K,56


In [3]:
#Height Column
def clean_Height(x):
    l=[]
    s=1
    if isinstance(x,str):
        l=x.split("'")
        i=int(l[0])
        j=int(l[1])
        s=((i*12)+j)/12
    return(s)


all_fifa_data_df['Height'] = all_fifa_data_df['Height'].apply(clean_Height)
all_fifa_data_df['Height'].fillna((all_fifa_data_df['Height'].mean()), inplace = True)

In [4]:
#Weight Column
def clean_Weight(x):
    
    if isinstance(x,str):
        return(x.replace('lbs', ''))
    return(x)

all_fifa_data_df['Weight'] = all_fifa_data_df['Weight'].apply(clean_Weight).astype('float')
all_fifa_data_df['Weight'].fillna((all_fifa_data_df['Weight'].mean()), inplace = True)

In [5]:
all_fifa_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 89 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        18207 non-null  int64  
 1   Name                      18207 non-null  object 
 2   Age                       18207 non-null  int64  
 3   Photo                     18207 non-null  object 
 4   Nationality               18207 non-null  object 
 5   Flag                      18207 non-null  object 
 6   Overall                   18207 non-null  int64  
 7   Potential                 18207 non-null  int64  
 8   Club                      18207 non-null  object 
 9   Club Logo                 18207 non-null  object 
 10  Value                     18207 non-null  object 
 11  Wage                      18207 non-null  object 
 12  Special                   18207 non-null  int64  
 13  Preferred Foot            18159 non-null  object 
 14  Intern

In [6]:
all_fifa_data_df.columns

Index(['ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag', 'Overall',
       'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiv

In [7]:
# #  drop nulls
# all_fifa_data_df.dropna(how='any', inplace = True)

In [8]:
le = LabelEncoder()
all_fifa_data_df['Work Rate'] = le.fit_transform(all_fifa_data_df['Work Rate'])

In [9]:
all_fifa_data_df = all_fifa_data_df.drop(['ID', 'Name','Photo', 'Nationality', 'Flag','Overall',
       'Potential','Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation','Body Type', 'Real Face', 
        'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until','Release Clause',
        'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB'], axis=1)
# add height and weight after conversion to see if accuracy imporves. 
# drop nulls
all_fifa_data_df.dropna(how='any', inplace = True)

In [10]:
all_fifa_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18159 entries, 0 to 18206
Data columns (total 42 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Age              18159 non-null  int64  
 1   Weak Foot        18159 non-null  float64
 2   Skill Moves      18159 non-null  float64
 3   Work Rate        18159 non-null  int32  
 4   Position         18159 non-null  object 
 5   Height           18159 non-null  float64
 6   Weight           18159 non-null  float64
 7   Crossing         18159 non-null  float64
 8   Finishing        18159 non-null  float64
 9   HeadingAccuracy  18159 non-null  float64
 10  ShortPassing     18159 non-null  float64
 11  Volleys          18159 non-null  float64
 12  Dribbling        18159 non-null  float64
 13  Curve            18159 non-null  float64
 14  FKAccuracy       18159 non-null  float64
 15  LongPassing      18159 non-null  float64
 16  BallControl      18159 non-null  float64
 17  Acceleration

In [11]:
# Set features. This will also be used as x values.
X = all_fifa_data_df.drop(['Position'], axis=1)
y = all_fifa_data_df["Position"]
print(X.shape, y.shape)


(18159, 41) (18159,)


In [12]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18159 entries, 0 to 18206
Data columns (total 41 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Age              18159 non-null  int64  
 1   Weak Foot        18159 non-null  float64
 2   Skill Moves      18159 non-null  float64
 3   Work Rate        18159 non-null  int32  
 4   Height           18159 non-null  float64
 5   Weight           18159 non-null  float64
 6   Crossing         18159 non-null  float64
 7   Finishing        18159 non-null  float64
 8   HeadingAccuracy  18159 non-null  float64
 9   ShortPassing     18159 non-null  float64
 10  Volleys          18159 non-null  float64
 11  Dribbling        18159 non-null  float64
 12  Curve            18159 non-null  float64
 13  FKAccuracy       18159 non-null  float64
 14  LongPassing      18159 non-null  float64
 15  BallControl      18159 non-null  float64
 16  Acceleration     18159 non-null  float64
 17  SprintSpeed 

In [13]:
all_fifa_data_df['Work Rate']

0        2
1        2
2        0
3        2
4        0
        ..
18202    2
18203    2
18204    2
18205    2
18206    2
Name: Work Rate, Length: 18159, dtype: int32

In [14]:
# Split data into training and testing groups and scale data 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [15]:
# Fit the data into model
rfm = RandomForestClassifier(n_estimators=250)
rfm.fit(X_train, y_train)

RandomForestClassifier(n_estimators=250)

In [16]:
sorted(zip(rfm.feature_importances_, X.columns), reverse=True)

# taking in all the values base on value combinations it is predicting what the ideal position would be. 


[(0.04459403382741017, 'Crossing'),
 (0.04253672026081254, 'SlidingTackle'),
 (0.03720796712715987, 'StandingTackle'),
 (0.036663507908819816, 'Positioning'),
 (0.036372882469102044, 'HeadingAccuracy'),
 (0.03257720008377686, 'Finishing'),
 (0.03208300528254548, 'Marking'),
 (0.03174056273269786, 'Interceptions'),
 (0.031503746425674925, 'Dribbling'),
 (0.030208084862246937, 'GKPositioning'),
 (0.02973796795793398, 'LongPassing'),
 (0.028870594509822503, 'GKHandling'),
 (0.02848179545793043, 'GKDiving'),
 (0.028437319081764483, 'Vision'),
 (0.02590209862511831, 'GKKicking'),
 (0.025069967327729772, 'SprintSpeed'),
 (0.02473672976005546, 'Acceleration'),
 (0.024581034170732832, 'GKReflexes'),
 (0.024293082974298642, 'Strength'),
 (0.023552064549591772, 'FKAccuracy'),
 (0.02285207391753661, 'LongShots'),
 (0.022664779587600915, 'BallControl'),
 (0.02233437674784508, 'Curve'),
 (0.02215926180460218, 'Volleys'),
 (0.02189300110904665, 'Penalties'),
 (0.021613653798946548, 'ShortPassing'),


In [17]:
predictions = rfm.predict(X_test)
base_train_accuracy = round(rfm.score(X_train, y_train)*100,3)
base_test_accuracy = round(rfm.score(X_test, y_test)*100,3)
print(f"Training Data Score: {rfm.score(X_train, y_train)}")
print(f"Testing Data Score: {rfm.score(X_test, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.5281938325991189


In [18]:
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

         CAM       0.45      0.47      0.46       252
          CB       0.52      0.89      0.66       421
         CDM       0.44      0.58      0.50       248
          CF       0.00      0.00      0.00        22
          CM       0.43      0.79      0.56       343
          GK       1.00      1.00      1.00       498
         LAM       0.00      0.00      0.00         6
          LB       0.46      0.56      0.51       326
         LCB       0.28      0.03      0.06       161
         LCM       0.00      0.00      0.00       103
         LDM       0.00      0.00      0.00        56
          LF       0.00      0.00      0.00         7
          LM       0.24      0.25      0.25       271
          LS       0.00      0.00      0.00        47
          LW       0.00      0.00      0.00       115
         LWB       0.00      0.00      0.00        20
         RAM       0.00      0.00      0.00         4
          RB       0.45    

C:\Users\alexv\anaconda3\envs\PythonData\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\alexv\anaconda3\envs\PythonData\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\alexv\anaconda3\envs\PythonData\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


for CAM position 

precision = accuracy 
recall = false positives probability 
f1-score = you weight harmonic mean of precision
support = how many rows that falls in the bucket. 

In [19]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)
# display(cm)
# print(y_test.shape)
# print(y_test.shape)
y_test
predictions

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(cm, index=["Actual 1", "Actual 2", "Actual 3", 
                                "Actual 4", "Actual 5", "Actual 6", 
                                "Actual 7", "Actual 8", "Actual 9",
                                "Actual 10", "Actual 11", "Actual 12", 
                                "Actual 13", "Actual 14", "Actual 15", 
                                "Actual 16", "Actual 17", "Actual 18", 
                                "Actual 19", "Actual 20", "Actual 21",
                                "Actual 22", "Actual 23", "Actual 24",
                                "Actual 25", "Actual 26", "Actual 27", "Actual 28"], 
                     columns=["Predicted 1", "Predicted 2", "Predicted 3", 
                                "Predicted 4", "Predicted 5", "Predicted 6", 
                                "Predicted 7", "Predicted 8", "Predicted 9",
                                "Predicted 10", "Predicted 11", "Predicted 12", 
                                "Predicted 13", "Predicted 14", "Predicted 15", 
                                "Predicted 16", "Predicted 17", "Predicted 18", 
                                "Predicted 19", "Predicted 20", "Predicted 21",
                                "Predicted 22", "Predicted 23", "Predicted 24",
                                "Predicted 25", "Predicted 26", "Predicted 27", "Predicted 28"])
display(cm_df)

,Predicted 1,Predicted 2,Predicted 3,Predicted 4,Predicted 5,Predicted 6,Predicted 7,Predicted 8,Predicted 9,Predicted 10,...,Predicted 19,Predicted 20,Predicted 21,Predicted 22,Predicted 23,Predicted 24,Predicted 25,Predicted 26,Predicted 27,Predicted 28
Actual 1,118,1,2,0,52,1,0,0,0,0,...,0,0,0,0,31,0,0,0,24,0
Actual 2,0,375,15,0,3,0,0,10,5,0,...,6,0,0,0,0,0,0,0,0,0
Actual 3,0,25,145,0,63,0,0,8,1,1,...,0,0,0,0,0,0,0,0,0,0
Actual 4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,17,0
Actual 5,18,1,38,0,271,0,0,1,0,1,...,0,0,0,0,4,0,0,0,1,0
Actual 6,0,0,0,0,0,498,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Actual 7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
Actual 8,0,16,7,0,16,0,0,184,1,0,...,0,0,0,0,1,0,0,0,0,0
Actual 9,0,123,9,0,2,0,0,8,5,0,...,4,0,0,0,0,0,0,0,0,0
Actual 10,13,4,28,0,44,0,0,1,0,0,...,0,1,0,0,6,0,0,0,2,0


In [20]:
acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 1,Predicted 2,Predicted 3,Predicted 4,Predicted 5,Predicted 6,Predicted 7,Predicted 8,Predicted 9,Predicted 10,...,Predicted 19,Predicted 20,Predicted 21,Predicted 22,Predicted 23,Predicted 24,Predicted 25,Predicted 26,Predicted 27,Predicted 28
Actual 1,118,1,2,0,52,1,0,0,0,0,...,0,0,0,0,31,0,0,0,24,0
Actual 2,0,375,15,0,3,0,0,10,5,0,...,6,0,0,0,0,0,0,0,0,0
Actual 3,0,25,145,0,63,0,0,8,1,1,...,0,0,0,0,0,0,0,0,0,0
Actual 4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,17,0
Actual 5,18,1,38,0,271,0,0,1,0,1,...,0,0,0,0,4,0,0,0,1,0
Actual 6,0,0,0,0,0,498,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Actual 7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
Actual 8,0,16,7,0,16,0,0,184,1,0,...,0,0,0,0,1,0,0,0,0,0
Actual 9,0,123,9,0,2,0,0,8,5,0,...,4,0,0,0,0,0,0,0,0,0
Actual 10,13,4,28,0,44,0,0,1,0,0,...,0,1,0,0,6,0,0,0,2,0


Accuracy Score : 0.5281938325991189
Classification Report
              precision    recall  f1-score   support

         CAM       0.45      0.47      0.46       252
          CB       0.52      0.89      0.66       421
         CDM       0.44      0.58      0.50       248
          CF       0.00      0.00      0.00        22
          CM       0.43      0.79      0.56       343
          GK       1.00      1.00      1.00       498
         LAM       0.00      0.00      0.00         6
          LB       0.46      0.56      0.51       326
         LCB       0.28      0.03      0.06       161
         LCM       0.00      0.00      0.00       103
         LDM       0.00      0.00      0.00        56
          LF       0.00      0.00      0.00         7
          LM       0.24      0.25      0.25       271
          LS       0.00      0.00      0.00        47
          LW       0.00      0.00      0.00       115
         LWB       0.00      0.00      0.00        20
         RAM       0.00

C:\Users\alexv\anaconda3\envs\PythonData\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\alexv\anaconda3\envs\PythonData\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\alexv\anaconda3\envs\PythonData\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
